# Homework 2: Filling and Visualizing Missing Data

Shown below are some ways to visualise the given dataset, and then visualise how well do a few filling-in methods work on the given modified dataset.

## Initialising the Data

We first read in the data, and perform some operations to make it useful.

Notably, we see that the data contains records which have two "category" labels, combining which we can get a single label for each record. This will help us group data in the later part of the notebook

In [1]:
import numpy as np
import pandas as pd
from bokeh.layouts import column, row
from bokeh.io import output_notebook, push_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, Select, CustomJS, CustomJSFilter, BooleanFilter, HoverTool
from bokeh.palettes import Category20

from ipywidgets import interact, Dropdown, HBox, VBox, SelectMultiple, FloatSlider
from IPython.display import display

output_notebook()

Loading BokehJS ...

The main pre-processing performed on the datasets is:

- Making the 'UniqueGroup' variable to specify the group for each record
- Adding a 'color' variable, assigned group-wise

In [2]:
all_data = pd.read_csv("Datasets/Wholesale_customers_data.csv")
all_data['Channel'] = all_data['Channel'].astype(str)
all_data['Region'] = all_data['Region'].astype(str)

mod_data = pd.read_csv("Datasets/Wholesale_customers_data_missing.csv")
mod_data['Channel'] = all_data['Channel'].astype(str)
mod_data['Region'] = all_data['Region'].astype(str)

usable_column_names = list(all_data.columns.values[2:])
num_rows = len(all_data.index.values)

for col_name in usable_column_names:
    all_data[col_name] = all_data[col_name].astype(dtype=np.float32)
    mod_data[col_name] = mod_data[col_name].astype(dtype=np.float32)

all_data['UniqueGroup'] = [(all_data['Channel'].values[i] + all_data['Region'].values[i]) for i in range(num_rows)]
mod_data['UniqueGroup'] = [(mod_data['Channel'].values[i] + mod_data['Region'].values[i]) for i in range(num_rows)]

all_data.sort_values(by="UniqueGroup",inplace=True)
mod_data.sort_values(by="UniqueGroup",inplace=True)

set_colors = {
                "11":Category20[20][0],
                "12":Category20[20][2],
                "13":Category20[20][4],
                "21":Category20[20][8],
                "22":Category20[20][6],
                "23":Category20[20][16]
            }

all_data['colors'] = [ set_colors[i] for i in all_data['UniqueGroup']]
mod_data['colors'] = [ set_colors[i] for i in all_data['UniqueGroup']]

## Visualisation 1: Two-Axis Scatter Plot over Numeric Variables

We will use this kind of a scatter plot to visualise the relationship between two numeric variables in the given dataset, grouped by the categor each record belongs to. This gives a quick overview of the data, and if a **strong** trend exists, it will be clearly visible. Color information is used to denote the groups.

Two plots are showed side by side, and each plot has individual control over what variable is shown on each axis. Using this, 4 variables at a time can be visualised.

In [3]:
scatter_data = mod_data.copy()
scatter_data['left_x_data'] = mod_data[usable_column_names[0]]
scatter_data['left_y_data'] = mod_data[usable_column_names[1]]
scatter_data['right_x_data'] = mod_data[usable_column_names[2]]
scatter_data['right_y_data'] = mod_data[usable_column_names[3]]

scatter_data_CDS = ColumnDataSource(scatter_data)


A **Hover** tool is provided to enable the user to view the details of a datapoint in the chart

In [4]:
scatter_tools = "lasso_select,box_select,box_zoom,wheel_zoom,reset,help"
hover_tool = HoverTool(tooltips=[
                (usable_column_names[0],"@"+usable_column_names[0]),
                (usable_column_names[1],"@"+usable_column_names[1]),
                (usable_column_names[2],"@"+usable_column_names[2]),
                (usable_column_names[3],"@"+usable_column_names[3]),
                (usable_column_names[4],"@"+usable_column_names[4]),
                (usable_column_names[5],"@"+usable_column_names[5])
])

scatter_figure_left = figure(plot_width=450,plot_height=400,tools=[hover_tool,scatter_tools])
scatter_handle_left = scatter_figure_left.circle(x='left_x_data',y='left_y_data',fill_color='colors',line_color='colors',legend='UniqueGroup',hover_fill_color="#000000",source=scatter_data_CDS)

scatter_figure_right = figure(plot_width=450,plot_height=400,tools=[hover_tool,scatter_tools])
scatter_handle_right = scatter_figure_right.circle(x='right_x_data',y='right_y_data',fill_color='colors',line_color='colors',legend='UniqueGroup',hover_fill_color="#000000",source=scatter_data_CDS)




**Dropdown menus** provide full flexibility to the user for selecting and assigning a variable to an axis of the chart. This selection is completely independent. 

The **Multi-Select** allows the user to view any and/or all groups at once, helping to focus on the important data while hiding the unnecessary portions.

In [5]:
left_x_select = Dropdown(options=usable_column_names,value=usable_column_names[0],description="X Data (Left)")
left_y_select = Dropdown(options=usable_column_names,value=usable_column_names[1],description="Y Data (Left)")
right_x_select = Dropdown(options=usable_column_names,value=usable_column_names[2],description="X Data (Right)")
right_y_select = Dropdown(options=usable_column_names,value=usable_column_names[3],description="Y Data (Right)")

group_mult_select = SelectMultiple(options=list(all_data.UniqueGroup.unique()),value=list(all_data.UniqueGroup.unique()),description="Selected Groups")

scatter_box = HBox([VBox([left_x_select,left_y_select]) , VBox([right_x_select,right_y_select]) , group_mult_select])


def scatterSelectCallback(change):

    nan = float('nan')
    
    if(change['type'] == 'change'):
        
        make_nan_boolean = [False if (ug in group_mult_select.value) else True for ug in scatter_data_CDS.data['UniqueGroup']]
        
        scatter_data_CDS.data['left_x_data'] = np.copy(scatter_data_CDS.data[left_x_select.value])
        scatter_data_CDS.data['left_y_data'] = np.copy(scatter_data_CDS.data[left_y_select.value])
        scatter_data_CDS.data['right_x_data'] = np.copy(scatter_data_CDS.data[right_x_select.value])
        scatter_data_CDS.data['right_y_data'] = np.copy(scatter_data_CDS.data[right_y_select.value])
        
        scatter_data_CDS.data['left_x_data'][make_nan_boolean] = nan
        scatter_data_CDS.data['left_y_data'][make_nan_boolean] = nan
        scatter_data_CDS.data['right_x_data'][make_nan_boolean] = nan
        scatter_data_CDS.data['right_y_data'][make_nan_boolean] = nan
        
        push_notebook()

        

left_x_select.observe(scatterSelectCallback,names="value")
left_y_select.observe(scatterSelectCallback,names="value")
right_x_select.observe(scatterSelectCallback,names="value")
right_y_select.observe(scatterSelectCallback,names="value")
        
group_mult_select.observe(scatterSelectCallback,names="value")


In [6]:
scatter_handle = show(row(scatter_figure_left,scatter_figure_right) , notebook_handle=True)

In [7]:
display(scatter_box)

Included in the plot is a **lasso** tool, which is helpful in selecting points arbitrarily. Notice that selection is linked across the two plots.

## Visualisation 2: Boxplot for visualising extents

The next plot is a standard boxplot, which shows the range of a particular variable across the groups, along with other single-dimension statistics.

In the plot that follows, the items shown per group are:
- *Max* and *Min* values, denoted by the **Vertical Line endpoints**
- *10th and 90th Quantile* values, denoted by the bottom and top edges of the **Vertical Bar**
- *Mean* and *Median* values, denoted by the **Circle** and **Triangle** respectively

In [8]:
unique_group_list = mod_data.UniqueGroup.unique()

col_mean = np.zeros((len(unique_group_list)+1 , len(usable_column_names)))
col_median = np.zeros((len(unique_group_list)+1 , len(usable_column_names)))
col_max = np.zeros((len(unique_group_list)+1 , len(usable_column_names)))
col_min = np.zeros((len(unique_group_list)+1 , len(usable_column_names)))
col_upperq = np.zeros((len(unique_group_list)+1 , len(usable_column_names)))
col_lowerq = np.zeros((len(unique_group_list)+1 , len(usable_column_names)))


for i in range(len(usable_column_names)):
    
    col_name = usable_column_names[i]
    for j in range(len(unique_group_list)):
        
        group_name = unique_group_list[j]
        
        series = mod_data.loc[mod_data['UniqueGroup']==group_name][col_name]
        
        col_mean[j][i] = series.mean()
        col_median[j][i] = series.median()
        col_max[j][i] = series.max()
        col_min[j][i] = series.min()
        col_upperq[j][i] = series.quantile(q=0.90 , interpolation='nearest')
        col_lowerq[j][i] = series.quantile(q=0.10 , interpolation='nearest')
        
        
    j = len(unique_group_list)
    
    series = mod_data[col_name]
    
    col_mean[j][i] = series.mean()
    col_median[j][i] = series.median()
    col_max[j][i] = series.max()
    col_min[j][i] = series.min()
    col_upperq[j][i] = series.quantile(q=0.90 , interpolation='nearest')
    col_lowerq[j][i] = series.quantile(q=0.10 , interpolation='nearest')
    

all_metrics_matrix = np.concatenate((col_mean[...,None],col_median[...,None],col_max[...,None],col_min[...,None],col_upperq[...,None],col_lowerq[...,None]),axis=2)

type_list = ["Mean","Median","Max","Min","Upper_Quartile","Lower_Quartile"]
id_list = usable_column_names

boxplot_data_dict = {}
for i in range(len(type_list)):
    for j in range(len(id_list)):
        boxplot_data_dict["{}_{}".format(str(id_list[j]),type_list[i])] = all_metrics_matrix[:,j,i]
        
boxplot_data_dict['colors'] = [ set_colors[i] for i in unique_group_list] + ["#000000"]

boxplot_data_dict['curr_mean'] = np.copy(boxplot_data_dict[usable_column_names[0]+"_Mean"])
boxplot_data_dict['curr_median'] = np.copy(boxplot_data_dict[usable_column_names[0]+"_Median"])
boxplot_data_dict['curr_max'] = np.copy(boxplot_data_dict[usable_column_names[0]+"_Max"])
boxplot_data_dict['curr_min'] = np.copy(boxplot_data_dict[usable_column_names[0]+"_Min"])
boxplot_data_dict['curr_upperq'] = np.copy(boxplot_data_dict[usable_column_names[0]+"_Upper_Quartile"])
boxplot_data_dict['curr_lowerq'] = np.copy(boxplot_data_dict[usable_column_names[0]+"_Lower_Quartile"])

boxplot_data_dict['x_values'] = list(unique_group_list) + ["Overall"]
        
boxplot_data = pd.DataFrame(data=boxplot_data_dict)

In [9]:
boxplot_data_CDS = ColumnDataSource(boxplot_data)
boxplot_x_axis = list(unique_group_list) + ["Overall"]

boxplot_figure = figure(x_range = boxplot_x_axis , plot_width=600, plot_height=400)

boxplot_figure.segment(x0='x_values' , y0='curr_max' , x1='x_values' , y1='curr_min' , line_color='colors' , line_width=3 , source=boxplot_data_CDS)
boxplot_figure.vbar(x='x_values' , bottom='curr_lowerq' , top='curr_upperq' , width=0.8 , fill_color='colors' , line_color='colors' , source=boxplot_data_CDS)
boxplot_figure.circle(x='x_values' , y='curr_mean' , fill_color="#FFFFFF" , line_color="#000000" , fill_alpha=0.6, size=15 , legend="Mean", source=boxplot_data_CDS)
boxplot_figure.triangle(x='x_values' , y='curr_median' , fill_color="#FFFFFF" , line_color="#000000" , fill_alpha=0.6, size=15 , legend="Median", source=boxplot_data_CDS)

GlyphRenderer(id='a536984c-3053-4482-b4b0-170f2eb579b9', ...)

In [10]:
boxplot_select = Dropdown(options=usable_column_names,value=usable_column_names[0],description="Attribute")

def boxplotCallback(change):
    
    if(change['type']=='change'):
        
        boxplot_data_CDS.data['curr_mean'] = np.copy(boxplot_data_CDS.data[boxplot_select.value+"_Mean"])
        boxplot_data_CDS.data['curr_median'] = np.copy(boxplot_data_CDS.data[boxplot_select.value+"_Median"])
        boxplot_data_CDS.data['curr_max'] = np.copy(boxplot_data_CDS.data[boxplot_select.value+"_Max"])
        boxplot_data_CDS.data['curr_min'] = np.copy(boxplot_data_CDS.data[boxplot_select.value+"_Min"])
        boxplot_data_CDS.data['curr_upperq'] = np.copy(boxplot_data_CDS.data[boxplot_select.value+"_Upper_Quartile"])
        boxplot_data_CDS.data['curr_lowerq'] = np.copy(boxplot_data_CDS.data[boxplot_select.value+"_Lower_Quartile"])
        
        push_notebook()
        
        
boxplot_select.observe(boxplotCallback,names="value")

In [11]:
boxplot_handle = show(boxplot_figure,notebook_handle = True)

In [12]:
display(boxplot_select)

The **Dropdown menu** can be used to select the attribute for which ranges and other data is desired to be visualised. Notice that the right-most bar is the average for the whole dataset, which is conveniently placed to understand each group's data with respect to the dataset as a whole. Colors used in the previous chart are replicated here to denote continuity.

The X-Axis labels denote the group, abbreviated as *{Channel}{Region}*

## Visualisation 3: Plotting Predicted values against the Actual value

Finally, we make a chart to show the predicted values (calculated using various methods) and plot them against the actual value. This is useful to see what methods works well, if any. The chart is also made interactive with an input parameter that can be used to tweak the results of the prediction methods

We first see what is the missing data

In [13]:
mod_is_nan = mod_data.isnull()

counter = 0
for col_name in usable_column_names:
    counter += len(list(mod_is_nan.index[mod_is_nan[col_name]]))

actual_value = []
column_name = []
orig_index = []
unique_group = []

for col_name in usable_column_names:
    index_list = list(mod_is_nan.index[mod_is_nan[col_name]])
    
    for id in index_list:
        column_name.append(col_name)
        orig_index.append(id)
        unique_group.append(mod_data['UniqueGroup'][id])
        actual_value.append(all_data[col_name][id])

temp_dict = {"Column_Name":column_name , "Orig_Index": orig_index , "Actual_Value":actual_value, "Unique_Group":unique_group}
missing_data = pd.DataFrame(data=temp_dict)

print(missing_data)

    

   Actual_Value       Column_Name  Orig_Index Unique_Group
0        6036.0              Milk         180           13
1           3.0           Grocery          75           13
2        6536.0           Grocery         172           13
3         761.0  Detergents_Paper         225           11
4         523.0  Detergents_Paper         370           23
5        1153.0        Delicassen         274           13


There are a total of 6 missing values in the dataset, as shown above.

In [14]:
good_data = mod_data.copy()
good_data.sort_index(inplace=True)
good_data.drop(good_data.index[list(missing_data['Orig_Index'])],inplace=True)

We use two helper function for truncating and normalizing the data. Truncating is done to avoid including very high or very low values (which may be outliers) in the calculations. Normalisation is done to make the data more useful when comparing and combining attributes.

In [15]:
def get_truncated_data(cutoff_ratio,ug=None):
    truncated_data = good_data.copy().sort_index()
    if(ug is not None):
        truncated_data = truncated_data.loc[truncated_data['UniqueGroup']==ug]
    
    upper_limit = truncated_data[usable_column_names].quantile(q=cutoff_ratio,interpolation='nearest').values
    lower_limit = truncated_data[usable_column_names].quantile(q=1-cutoff_ratio,interpolation='nearest').values
    
    for i in range(len(upper_limit)):
        
        orig_values = truncated_data[usable_column_names[i]].values
        orig_values[orig_values>upper_limit[i]] = upper_limit[i]
        orig_values[orig_values<lower_limit[i]] = lower_limit[i]
        truncated_data[usable_column_names[i]] = orig_values
        
    return truncated_data


def get_normalized_data(input_data):
    norm_data = ((input_data - input_data.min()) / (input_data.max() - input_data.min()))*2 - 1
    return norm_data


The following four functions are very simple in design: Given a missing value, it is filled in with either the median or the mode of the corresponding column; additionally, it supports grouping, in which case the dataset for this mean/median calculation is pruned to retain only records belonging to the same group as that of the missing data.

In [16]:
def simple_average(cutoff_ratio,row_num):
    col_name = missing_data.loc[row_num]['Column_Name']
    truncated_data = get_truncated_data(cutoff_ratio)
    return truncated_data[col_name].mean()

def grouped_average(cutoff_ratio,row_num):
    col_name = missing_data.loc[row_num]['Column_Name']
    ug = mod_data.loc[row_num]['UniqueGroup']
    truncated_data = get_truncated_data(cutoff_ratio,ug)
    return truncated_data.loc[truncated_data['UniqueGroup']==ug][col_name].mean()

def simple_median(cutoff_ratio,row_num):
    col_name = missing_data.loc[row_num]['Column_Name']
    truncated_data = get_truncated_data(cutoff_ratio)
    return truncated_data[col_name].median()

def grouped_median(cutoff_ratio,row_num):
    col_name = missing_data.loc[row_num]['Column_Name']
    ug = mod_data.loc[row_num]['UniqueGroup']
    truncated_data = get_truncated_data(cutoff_ratio,ug)
    return truncated_data.loc[truncated_data['UniqueGroup']==ug][col_name].median()
                                                                                                  

The *weighted()* function essentially calculates the similarity of the missing data record, with all the other existing records. It then uses this similarity to perform a weighted vote on the missing data value. In practice, it is equivalent to the **Collaborative Filtering** approach used in Recommender Systems.

Like the previous functions, this one supports groupin in the exact same manner

In [17]:
def weighted(cutoff_ratio,row_num,isGrouped=False):
    
    col_name = missing_data['Column_Name'][row_num]
    orig_index = missing_data['Orig_Index'][row_num]
    
    if(isGrouped==True):
        ug = mod_data.loc[orig_index]['UniqueGroup']
        trunc_data = get_truncated_data(cutoff_ratio,ug)
    else:
        trunc_data = get_truncated_data(cutoff_ratio)
        
    norm_data = get_normalized_data(trunc_data[usable_column_names])
    
    test_vector = mod_data.loc[orig_index][usable_column_names]
    test_vector = ((test_vector - trunc_data[usable_column_names].min()) / (trunc_data[usable_column_names].max() - trunc_data[usable_column_names].min()))*2 - 1
    
    test_vector.drop(col_name,inplace=True)
    shrunk_norm_data = norm_data.drop(col_name,axis=1)
    
    dist_vec = ((test_vector-shrunk_norm_data)**2).sum(axis=1)
    weight_vec = dist_vec / dist_vec.sum()
    
    norm_ans = (weight_vec*norm_data[col_name]).sum()
    
    final_ans = ((norm_ans + 1)/2.0) * (trunc_data[col_name].max() - trunc_data[col_name].min()) + trunc_data[col_name].min()
    
    return final_ans
        
        

In [18]:
predicted_vals = []

predicted_vals.append(simple_average(0.85,0))
predicted_vals.append(grouped_average(0.85,0))
predicted_vals.append(simple_median(0.85,0))
predicted_vals.append(grouped_median(0.85,0))
predicted_vals.append(weighted(0.85,0,isGrouped=False))
predicted_vals.append(weighted(0.85,0,isGrouped=True))

pred_colors = [Category20[20][0],Category20[20][1],Category20[20][2],Category20[20][3],Category20[20][4],Category20[20][5]]

x_vals = [1 for i in range(len(predicted_vals))]

pred_legend = ['Simple Average','Grouped Average','Simple Median','Grouped Median','Weighted Vote','Grouped Weighted Vote']

filling_data_dict = {'x_values': x_vals, 'y_values': predicted_vals, 'colors': pred_colors, 'legend': pred_legend}
filling_data_CDS = ColumnDataSource(filling_data_dict)


actual_x = [1]
actual_val = [missing_data.loc[0]['Actual_Value']]
actual_color = ["#000000"]

actual_data_dict = {'x_values': actual_x, 'y_values': actual_val, 'colors': actual_color}
actual_data_CDS = ColumnDataSource(actual_data_dict)

In [19]:
filling_figure = figure(plot_width = 800, plot_height = 400)
filling_figure.circle_cross(x='x_values',y='y_values',fill_color='colors',line_color='colors',size=30,fill_alpha=0.6,legend="Actual Value",source=actual_data_CDS)
filling_figure.circle(x='x_values',y='y_values',fill_color='colors',line_color='colors',size=20,fill_alpha=0.8,legend='legend',source=filling_data_CDS)


GlyphRenderer(id='5fd44273-86dd-45ee-8b31-267b9e2e5488', ...)

In [20]:
missing_options = list(np.arange(len(missing_data.index)))
filling_select = Dropdown(options=missing_options,value=missing_options[0],description="Missing Field #")

filling_slider = FloatSlider(value=0.85,min=0.51,max=1.00,step=0.01,description="Data: Quantile-Cutoff",continuous_update=False)

def fillingCallback(change):
    
    if(change['type']=='change'):
        
        num_row = int(filling_select.value)
        cutoff_ratio = float(filling_slider.value)
        predicted_vals = []

        predicted_vals.append(simple_average(cutoff_ratio,num_row))
        predicted_vals.append(grouped_average(cutoff_ratio,num_row))
        predicted_vals.append(simple_median(cutoff_ratio,num_row))
        predicted_vals.append(grouped_median(cutoff_ratio,num_row))
        predicted_vals.append(weighted(cutoff_ratio,num_row,isGrouped=False))
        predicted_vals.append(weighted(cutoff_ratio,num_row,isGrouped=True))
        
        filling_data_CDS.data['y_values'] = predicted_vals
        
        actual_data_CDS.data['y_values'] = [missing_data.loc[num_row]['Actual_Value']]
        
        push_notebook()
        
filling_select.observe(fillingCallback,names="value")
filling_slider.observe(fillingCallback,names="value")

In [21]:
show(filling_figure,notebook_handle=True)

In [22]:
display(HBox([filling_select,filling_slider]))

Using the Dropdown menu, one can select which missing value is being visualised.

Using the slider on the right, one can change the cutoff parameter to the data truncating function. This parameter controls what percentage data is to be used. For example, setting it to **0.9** means that the top and bottom **5%** of the data is ignored in the calculations. This is done to typically exlude potential outliers from the calculations.